In [13]:
import wandb
import numpy as np
import torch
import cv2 as cv
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from torch.optim import RMSprop
from torch.optim.lr_scheduler import StepLR
from data_loaders import CAERSDataset
from utils import get_transform, accuracy, accuracy_julia
from model import model_generator
from tqdm import tqdm
import os.path as osp
from keras.models import load_model
import pandas as pd
import os

In [2]:
model_path = '.\models\\faces_19_95.01295733262594.pth'
model = torch.load(model_path)


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") ## Se a maquina não tiver GPU o teste será rodado na CPU

test_dataset = CAERSDataset('data/CAER-S', 'data/test.txt', transforms=get_transform(train=False))

test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)


model.to(device)
loss = CrossEntropyLoss()
opt = RMSprop(model.parameters(), lr=3e-3)
lr_scheduler = StepLR(opt, 60, 0.4)

best_model_acc = 0.0
lowest_loss = 1
count_ep = 0

In [4]:
model.eval() ## modo teste e validação
running_loss = 0.0
test_outs, test_gts = [],[] ##malu

with torch.no_grad():
    with tqdm(total=len(test_dataloader), position=1, leave=True) as pbar_1:
        for data, label in iter(test_dataloader):
            face = data['face']

            preds = model(face.to(device))
            preds = preds.to('cpu')

            running_loss = loss(preds, label)
            test_outs.extend(torch.argmax(preds, dim=1))##malu
            test_gts.extend(label.to('cpu'))##malu
            pbar_1.update(1)
            #image =  cv.imread(face)
            #cv.imshow('teste',image)
            #cv.waitkey(0)
        
            
                   

    test_acc = accuracy_julia(test_outs,test_gts)
    #print(f'test :: acc {test_acc} :: loss {running_loss}')

100%|██████████| 1532/1532 [12:22<00:00,  2.06it/s]


In [26]:
def set_emotion(label):

    if label == '0' :
        emotion = 'Angry'
    elif label == '1':
        emotion = 'Disgust'
    elif label == '2':
        emotion = 'Fear'
    elif label == '3':
        emotion = 'Happy'
    elif label == '4':
        emotion = 'Neutral'
    elif label == '5':
        emotion = 'Sad'
    elif label == '6':
        emotion = 'Surprise'
    return emotion
    

In [64]:
value=0
emotion = ''
data = ('0','0','0','0','0','0','0')
#emotions = ["Angry", "Disgust","Fear", "Happy", "Neutral", "Sad", "Surprise"]
emotions = ['Angry', 'Disgust','Fear','Happy','Neutral','Sad','Surprise']
zero = [0, 0,0,0,0,0,0]
d = {'Angry':zero, 'Disgust':zero,'Fear':zero,'Happy':zero,'Neutral':zero,'Sad':zero,'Surprise':zero}
table= pd.DataFrame( data=d, index=emotions)
#table= pd.DataFrame(data=table1,index= emotions, columns= emotions)


#table.loc["Angry", 'Angry']= 'oi'

for i in range(len(test_outs)):
    

    preds = str(test_outs[i].numpy())
    gt = str(test_gts[i].numpy())
    emotion_Pred =str(set_emotion(preds))
    emotion_gt =str(set_emotion(gt))
    
    table.loc [emotion_gt,emotion_Pred ] = int(table.loc [emotion_gt,emotion_Pred ]) +1
    

print(table)



          Angry  Disgust  Fear  Happy  Neutral   Sad  Surprise
Angry      6385       50     1     75      179   118       193
Disgust      55     6772    11     36       37    27        63
Fear         13       32  6742     42       64    46        62
Happy         5       41     5   6804       75    20        51
Neutral      55       43     6    266     6463    66       102
Sad          18       58    13     75      106  6641        90
Surprise     17        9     8     52      114    45      6756


In [ ]:
for i in range(len(test_outs)):
    

    preds = str(test_outs[i].numpy())
    emption_Pred =str(set_emotion(preds))
    table.insert(int(test_gts[i]), emption_Pred, "hi", allow_duplicates=False )

In [5]:
import os
from urllib import request

def salvar_imagem(imagem, nome_arquivo, pasta):
    # Verifica se a pasta existe
    if not os.path.exists(pasta):
        # Cria a pasta se não existir
        os.makedirs(pasta)

    # Define o caminho completo para o arquivo
    caminho_arquivo = os.path.join(pasta, nome_arquivo)

    # Faz o download da imagem
    #request.urlretrieve(url, caminho_arquivo)
    cv.imwrite(caminho_arquivo,imagem)

    print(f'A imagem {nome_arquivo} foi salva em {pasta}.')

In [10]:
import matplotlib.pyplot as plt
#from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
#X, y = make_classification(random_state=0)
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
clf = SVC(random_state=0)
#clf.fit(test_outs,test_gts)

#predictions = clf.predict(test_outs)
cm = confusion_matrix(test_gts, test_outs, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=clf.classes_)
disp.plot()

plt.show()

AttributeError: 'SVC' object has no attribute 'classes_'